<a href="https://colab.research.google.com/github/ricky850/option/blob/master/Option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
files = os.listdir()
if 'weeklyovsv.csv' not in files:
  file_id = '1DNg3BaB2IJHkJPyYOJ2IK0nUF8U5Yzex'  #雲端硬碟檔案連結碼d/
  downloaded = drive.CreateFile({'id' : file_id})
  downloaded.GetContentFile('weeklyovsv.csv')

if 'weeklyskew.csv' not in files:
  file_id = '14_1NRu-FvYn0Yy9E_Gcbhn_TQLww0Yjq'  #雲端硬碟檔案連結碼d/
  downloaded = drive.CreateFile({'id' : file_id})
  downloaded.GetContentFile('weeklyskew.csv')

## ovsv檔EDA

In [23]:
df_v = pd.read_csv('weeklyovsv.csv')
df_v['date'] = pd.to_datetime([str(date)[:4]+'-'+str(date)[4:6]+'-'+str(date)[6:] for date in df_v['date']])
print('資料數量:', len(df_v))
df_v.head()

資料數量: 13182265


,Unnamed: 0,weeklycallvol,secid,weeklyputvol,weeklycallnum,weeklyputnum,weeklystockvol,date,weeklyvol
0,0,5000.0,5005,11000.0,39.0,52.0,9059.18,1996-01-11,0.438961
1,3,3000.0,5028,1000.0,31.0,36.0,7033.80,1996-01-11,0.090631
2,5,6100.0,5036,4500.0,32.0,28.0,35345.00,1996-01-11,0.097943
3,8,3100.0,5048,89300.0,28.0,29.0,31523.00,1996-01-11,0.720609
4,10,324400.0,5052,45400.0,54.0,56.0,11030.00,1996-01-11,0.119213


In [11]:
df_v.isnull().sum()

Unnamed: 0        0
weeklycallvol     0
secid             0
weeklyputvol      0
weeklycallnum     0
weeklyputnum      0
weeklystockvol    0
date              0
weeklyvol         0
dtype: int64

In [18]:
df_v['weeklyvol'].describe()[1:]

mean    0.236162
std     0.189927
min     0.000009
25%     0.079606
50%     0.193096
75%     0.351574
max     0.998670
Name: weeklyvol, dtype: float64

In [3]:
df_v.groupby('secid').size().describe()

count    7580.000000
mean     1739.085092
std      1969.395842
min         1.000000
25%       317.750000
50%       937.000000
75%      2503.250000
max      8024.000000
dtype: float64

In [4]:
df_v.groupby('date').size().describe()

count    8024.000000
mean     1642.854561
std       460.068729
min        73.000000
25%      1373.750000
50%      1701.000000
75%      2015.000000
max      2416.000000
dtype: float64

## skew檔EDA

In [24]:
df_p = pd.read_csv('weeklyskew.csv')
df_p['date'] = pd.to_datetime([str(date)[:4]+'-'+str(date)[4:6]+'-'+str(date)[6:] for date in df_p['date']])
print('資料數量:', len(df_p))
df_p.head()

資料數量: 9417754


,Unnamed: 0,skewcount,secid,weeklyskew,date
0,0,4,5036,0.015353,1996-01-11
1,1,2,5048,0.004091,1996-01-11
2,2,3,5052,0.028396,1996-01-11
3,3,6,5067,0.014649,1996-01-11
4,4,6,5068,-0.005197,1996-01-11


In [12]:
df_p.isnull().sum()

Unnamed: 0    0
skewcount     0
secid         0
weeklyskew    0
date          0
dtype: int64

In [20]:
df_p['weeklyskew'].describe()[1:]
# 有負值

mean    0.058308
std     0.067418
min    -1.379648
25%     0.026119
50%     0.042880
75%     0.070158
max     1.680941
Name: weeklyskew, dtype: float64

In [6]:
df_p.groupby('secid').size().describe()

count    6692.000000
mean     1407.315302
std      1756.434410
min         1.000000
25%       202.750000
50%       660.000000
75%      1908.750000
max      7997.000000
dtype: float64

In [7]:
df_p.groupby('date').size().describe()

count    8022.000000
mean     1173.990775
std       332.846695
min       314.000000
25%       917.000000
50%      1129.000000
75%      1427.000000
max      1954.000000
dtype: float64

## ovsv及skew比較

In [8]:
v_date = df_v.groupby('date').size().index
p_date = df_p.groupby('date').size().index
print('僅在ovsv檔 不在skew檔中的date:',set(v_date) - set(p_date))
print('僅在skew檔 不在ovsv檔中的date:',set(p_date) - set(v_date))

僅在ovsv檔 不在skew檔中的date: {Timestamp('2001-09-15 00:00:00'), Timestamp('2001-09-16 00:00:00')}
僅在skew檔 不在ovsv檔中的date: set()


In [9]:
v_sid = df_v.groupby('secid').size().index
p_sid = df_p.groupby('secid').size().index
print('共有 {} 個不同的secid'.format(len(set(v_sid)|set(p_sid))))
print('重疊之secid有:',len(set(v_sid)&set(p_sid)))
print('僅在ovsv檔 不在skew檔中的secid數量:',len(set(v_sid) - set(p_sid)))
print('僅在skew檔 不在ovsv檔中的secid數量:',len(set(p_sid) - set(v_sid)))

共有 7591 個不同的secid
重疊之secid有: 6681
僅在ovsv檔 不在skew檔中的secid數量: 899
僅在skew檔 不在ovsv檔中的secid數量: 11
